Connected to SI-0_100_2 (Python 3.10.12)

In [1]:
import numpy as np
import spikeinterface.full as si
import matplotlib.pyplot as plt
import elephant as elep
from ap_envelope import make_ap_envelope
%matplotlib auto

Using matplotlib backend: <object object at 0x7e3d407cf660>


In [ ]:
# cd /nas/Neuropixel_Recordings/AreaX-LMAN/Imp_07_03_2022/Recordings/Rec_13_03_2022_g0_from_7200_to_14400s/

In [ ]:
# cd /nas/Neuropixel_Recordings/AreaX-LMAN/Imp_29_11_2022/Recordings/Rec_2_30_11_2022_morning_nap_g0/

In [3]:
cd /nas/Neuropixel_Recordings/AreaX-LMAN/Imp_16_08_2022/Recordings/Rec_18_08_2022_g0/

/nas/Neuropixel_Recordings/AreaX-LMAN/Imp_16_08_2022/Recordings/Rec_18_08_2022_g0


/home/eduarda/.virtualenvs/SI-0_100_2/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
rec = si.read_spikeglx('.', stream_id='imec0.ap')

In [5]:
target_channels = ['imec0.ap#AP70', 'imec0.ap#AP80', 'imec0.ap#AP90']
start_time = 0*rec.get_sampling_frequency() 
end_frame = 50*rec.get_sampling_frequency() 

In [6]:
ps_rec = si.phase_shift(rec)

In [7]:
ps_channel_sliced = ps_rec.channel_slice(channel_ids=target_channels)

ps_rec_time_sliced = ps_channel_sliced.frame_slice(start_frame=start_time, end_frame=end_frame)

In [8]:
zero_centered = si.bandpass_filter(ps_rec_time_sliced,
                        freq_min=300,
                        freq_max=6000,
                    )

In [9]:
trace_rectified = si.rectify(zero_centered)

In [10]:
lp_freq = 500

lp_trace = si.gaussian_filter(trace_rectified, freq_min=None, freq_max=lp_freq)

In [11]:
FOI = 1000 # frequency of interest

margin_ms = 3000

ds_factor = int(rec.get_sampling_frequency() / FOI)

samp_period = rec.get_sampling_frequency()/ds_factor

ds = si.resample(lp_trace, samp_period, margin_ms=margin_ms)

In [12]:
ds_saved = ds.save(folder='/home/eduarda/python-related/github-repos/test_envelope1/', n_jobs=-1, chunk_duration='3s')

write_binary_recording with n_jobs = 40 and chunk_size = 3000


write_binary_recording:   0%|          | 0/17 [00:00<?, ?it/s]

In [13]:
def save_envelope(trace, fs, lowpass=500, ds=1000, zero_center=True, visualize_traces=True):    
    ds_factor = int(fs / ds)
    original_trace = trace.ravel()
    if zero_center:
        original_trace = original_trace-np.mean(original_trace) # Make sure is 0-centered
    # print('original', original_trace)
    
    # Rectify
    trace_rectified = np.abs(original_trace)
    
    # Low pass
    lp_trace = elep.signal_processing.butter(
        signal=trace_rectified,
        highpass_frequency=None,
        lowpass_frequency=lowpass,
        sampling_frequency=fs
    )

    # Downsample
    ds_trace = lp_trace[::ds_factor]
    real_ds_fs = fs/ds_factor

    if visualize_traces:
            
        plt.figure()
        
        plt.plot(np.linspace(0,1,
                             len(original_trace[:int(fs)])), 
                 original_trace[:int(fs)], c='blue')
        
        plt.plot(np.linspace(0,1,
                              len(trace_rectified[:int(fs)])), 
                  trace_rectified[:int(fs)], c='green')
        
        
        plt.plot(np.linspace(0,1,
                              len(ds_trace[:int(real_ds_fs)])), 
                  ds_trace[:int(real_ds_fs)], c='red')#
        
        plt.show()
    
    return real_ds_fs, ds_trace

In [14]:
ap_envelope = make_ap_envelope(ps_rec_time_sliced, ds_rate=FOI, lowpass_freq=lp_freq, margin_ms=margin_ms)


In [16]:
ds_saved2 = ds.save(folder='/home/eduarda/python-related/github-repos/test_envelope2/', n_jobs=-1, chunk_duration='3s')
ds_saved2_fs = ds_saved2.get_sampling_frequency()

write_binary_recording with n_jobs = 40 and chunk_size = 3000


write_binary_recording:   0%|          | 0/17 [00:00<?, ?it/s]

In [21]:
beg = 40
end = 45
target_channel = target_channels[1]


original_trace = zero_centered.frame_slice(start_frame=beg*zero_centered.get_sampling_frequency(), end_frame=end*zero_centered.get_sampling_frequency()).channel_slice([target_channel]).get_traces(return_scaled=True)
trace_rectified2 = trace_rectified.frame_slice(start_frame=beg*trace_rectified.get_sampling_frequency(), end_frame=end*trace_rectified.get_sampling_frequency()).channel_slice([target_channel]).get_traces(return_scaled=True)

ds_trace = ds.frame_slice(start_frame=beg*ds.get_sampling_frequency(), end_frame=end*ds.get_sampling_frequency()).channel_slice([target_channel]).get_traces(return_scaled=True)
ds_saved_trace  = ds_saved.frame_slice(start_frame=beg*ds_saved.get_sampling_frequency(), end_frame=end*ds_saved.get_sampling_frequency()).channel_slice([target_channel]).get_traces(return_scaled=True)

func_trace = ap_envelope.frame_slice(start_frame=beg*ap_envelope.get_sampling_frequency(), end_frame=end*ap_envelope.get_sampling_frequency()).channel_slice([target_channel]).get_traces(return_scaled=True)
ds_saved_trace2  = ds_saved2.frame_slice(start_frame=beg*ds_saved2.get_sampling_frequency(), end_frame=end*ds_saved2.get_sampling_frequency()).channel_slice([target_channel]).get_traces(return_scaled=True)

r, ds_trace2 = save_envelope(ps_rec_time_sliced.channel_slice([target_channel]).get_traces(return_scaled=True), fs=ps_rec_time_sliced.get_sampling_frequency(), ds=FOI, lowpass=lp_freq, visualize_traces=False)

In [22]:
f2, ax2 = plt.subplots()

ax2.plot(np.linspace(beg,end,
                    len(original_trace)), 
        original_trace, c='blue', label='original'
        # alpha=0.
        )

ax2.plot(np.linspace(beg,end,
                    len(trace_rectified2)), 
        trace_rectified2, c='green', label='rectified'
        # alpha=0.5
        ) 

ax2.plot(np.linspace(beg,end,
                    len(ds_trace)), 
                    ds_trace, 
                    c='red', 
                    label='not saved ds wo class') # not saved ds

ax2.plot(np.linspace(beg,end,
                    len(ds_saved_trace)),
                    ds_saved_trace, 
                    c='purple', label='saved ds wo class'
                    ) # saved ds

ax2.plot(np.linspace(beg,end,
                    len(func_trace)), 
        func_trace, c='grey', label='not saved ds w class') # mean shifted

ax2.plot(np.linspace(beg,end,
                    len(ds_saved_trace2)),
                    ds_saved_trace2, c='orange', label='saved ds w class') # saved ds

ax2.plot(np.linspace(beg,end,
                    len(ds_trace2[int(beg*r):int(end*r)])),
                    ds_trace2[int(beg*r):int(end*r)], 
                    c='yellow', 
                    label='old comp') # old computation

ax2.legend()